## Big Data Day 2 Afternoon Assignment

In this assignment, we will learn about machine learning with Dask. We will use the market basket dataset loaded below and cluster our data.

In [7]:
!pip install distributed>=2.4.0 --quiet

!pip install dask-ml --quiet
!pip install -U ipykernel --quiet

!pip install scikit-learn==0.23.2 # needed for dask_ml.cluster.KMeans
!python -m pip install "dask[distributed]" --upgrade # needed for dask_ml.cluster.KMeans

In [6]:
import dask.dataframe as dd
import numpy as np
import pandas as pd

In [7]:
market_basket = dd.read_csv('/content/Mall_Customers.csv')

In [8]:
market_basket.head()

,CustomerID,Gender,Age,Annual Income (k$),Spending Score (1-100)
0,1,Male,19,15,39
1,2,Male,21,15,81
2,3,Female,20,16,6
3,4,Female,23,16,77
4,5,Female,31,17,40


To cluster our data, we must first examine and process it. The first step is to check for missing data. Do this in the cell below. If there is missing data, drop all rows containing missing data.

In [9]:
# Answer below:
market_basket.isnull().sum().compute()


CustomerID                0
Gender                    0
Age                       0
Annual Income (k$)        0
Spending Score (1-100)    0
dtype: int64

Next, we will get rid of the customer ID column. Drop the column and assign the resulting dataframe to a new variable.

In [10]:
# Answer below:
df = market_basket.drop(columns=['CustomerID'])
df

,Gender,Age,Annual Income (k$),Spending Score (1-100)
npartitions=1,,,,
,object,int64,int64,int64
,...,...,...,...


We will now create a dummy variable from the gender variable.

In [11]:
# Answer below
df.dtypes

Gender                    object
Age                        int64
Annual Income (k$)         int64
Spending Score (1-100)     int64
dtype: object

In [12]:
df = dd.get_dummies(data=df.categorize(), columns=['Gender'], drop_first=True)

In [18]:
df.head()

,Age,Annual Income (k$),Spending Score (1-100),Gender_Female
0,19,15,39,0
1,21,15,81,0
2,20,16,6,1
3,23,16,77,1
4,31,17,40,1


In [33]:
df['Gender_Female'].min().compute()

0

Since the data has different scales, we will scale all columns using min max scaling. We will write our own min max scaling function since the minmax scaler will produce numpy arrays instead of dask arrays. Recall that min max scaling requires finding the min and the max. We subtract the min from each observation and divide by the difference between the max and the min. Complete the function below.

In [65]:
df.min().compute()

Age                       18
Annual Income (k$)        15
Spending Score (1-100)     1
Gender_Female              0
dtype: int64

In [66]:
# Answer below:

def dask_min_max(df):
    #Input: dask column
    min_array = df.min(axis=0)
    max_array = df.max(axis=0)
    scaled = (df - min_array) / (max_array-min_array)

    #Output: dask column that has been min max scaled
    return  scaled


In [67]:
dask_min_max(df).compute()

,Age,Annual Income (k$),Spending Score (1-100),Gender_Female
0,0.019231,0.000000,0.387755,0.0
1,0.057692,0.000000,0.816327,0.0
2,0.038462,0.008197,0.051020,1.0
3,0.096154,0.008197,0.775510,1.0
4,0.250000,0.016393,0.397959,1.0
...,...,...,...,...
195,0.326923,0.860656,0.795918,1.0
196,0.519231,0.909836,0.275510,1.0
197,0.269231,0.909836,0.744898,0.0
198,0.269231,1.000000,0.173469,0.0


Transform all columns using the scaler you wrote above.

In [68]:
# Answer below:

df_scaled = dask_min_max(df)


Import the dask clustering function and cluster the data. Use 3 clusters and print the centroids.

In [ ]:
# Answer below
from dask_ml.cluster import KMeans


In [70]:
model = KMeans(n_clusters=3)
model.fit(df_scaled)

KMeans(n_clusters=3)

In [72]:
model.cluster_centers_

array([[0.18515206, 0.393252  , 0.67489321, 0.        ],
       [0.38650412, 0.36270492, 0.51557945, 1.        ],
       [0.64316239, 0.38123862, 0.3031746 , 0.        ]])